##### Data preparation for insertion in the database

In [52]:
import pandas as pd
import numpy as np

files = {
    'name_basics': 'truncated_name.basics.tsv',
    'title_akas': 'truncated_title.akas.tsv',
    'title_basics': 'truncated_title.basics.tsv',
    'title_crew': 'truncated_title.crew.tsv',
    'title_episode': 'truncated_title.episode.tsv',
    'title_principals': 'truncated_title.principals.tsv',
    'title_ratings': 'truncated_title.ratings.tsv'
}

replacement_dict = {
    '\\N': None
}

dataframes = {key: pd.read_csv(files[key], sep = '\t', low_memory=False) for key in files}

print(dataframes)

{'name_basics':          nconst       primaryName birthYear deathYear  \
0     nm0000019  Federico Fellini      1920      1993   
1     nm0000030    Audrey Hepburn      1929      1993   
2     nm0000035      James Horner      1953      2015   
3     nm0000047      Sophia Loren      1934        \N   
4     nm0000053    Robert Mitchum      1917      1997   
...         ...               ...       ...       ...   
4031  nm9634588       Ioana Filip        \N        \N   
4032  nm9861435     Andrew Hannan        \N        \N   
4033  nm9861436         Ralph Eue        \N        \N   
4034  nm9928014       Guoren Wang        \N        \N   
4035  nm9928038     Wong Ji Cheng        \N        \N   

                         primaryProfession  \
0                    writer,director,actor   
1                       actress,soundtrack   
2     music_department,soundtrack,composer   
3                       actress,soundtrack   
4                actor,soundtrack,producer   
...                    

Preparing to add data to the Title table:

In [53]:
import random

title_df = pd.merge(dataframes['title_basics'], dataframes['title_ratings'], how='left', on='tconst')
title_df = title_df[['tconst', 'primaryTitle', 'titleType', 'startYear', 'endYear', 'runtimeMinutes', 'averageRating', 'numVotes', 'isAdult', 'img_url_asset']]
title_df.columns = ['Title_ID', 'Original_Title', 'Type', 'Start_Year', 'End_Year', 'Runtime', 'Average_Rating', 'Votes', 'isAdult', 'IMAGE']
title_df.fillna(random.randint(1, 10), inplace=True)
title_df.replace('\\N', None, inplace=True)

print(title_df)

      Title_ID            Original_Title       Type  Start_Year End_Year  \
0    tt0000929      Klebolin klebt alles      short        1990     None   
1    tt0000977               Mutterliebe      short        1990     None   
2    tt0034841                   Hen Hop      short        1994     None   
3    tt0040844      Crossroads of Laredo      short        1995     None   
4    tt0078006     Norman and the Killer      short        1991     None   
..         ...                       ...        ...         ...      ...   
495  tt0103054        Wer zweimal stirbt  tvEpisode        1991     None   
496  tt0103062  Tell Me That You Love Me  tvEpisode        1991     None   
497  tt0103071        Thanners neuer Job  tvEpisode        1991     None   
498  tt0103145      Two Teens and a Baby  tvEpisode        1992     None   
499  tt0103149    Tödliche Vergangenheit  tvEpisode        1991     None   

    Runtime  Average_Rating  Votes  isAdult  \
0      None             5.3   46.0      

Preparing for Alt_Title table:

In [54]:
alt_title_df = dataframes['title_akas'][['titleId', 'ordering', 'title', 'region']]
alt_title_df.columns = ['Title_FK', 'Ordering', 'Title_AKA', 'Region']
alt_title_df.fillna(random.randint(1, 100), inplace=True)
alt_title_df.replace('\\N', None, inplace=True)

alt_title_df.head()

/tmp/ipykernel_6222/2256535267.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alt_title_df.fillna(random.randint(1, 100), inplace=True)
/tmp/ipykernel_6222/2256535267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alt_title_df.replace('\\N', None, inplace=True)


,Title_FK,Ordering,Title_AKA,Region
0,tt0000929,1,Willys Streiche: Klebolin klebt alles,DE
1,tt0000929,2,Klebolin klebt alles,None
2,tt0000929,3,Klebolin klebt alles,DE
3,tt0000977,1,Mutterliebe,None
4,tt0000977,2,Mutterliebe,DE


Episode table:

In [55]:
episode_df = dataframes['title_episode'][['tconst', 'parentTconst', 'seasonNumber', 'episodeNumber']]
episode_df.columns = ['Title_ID', 'parentTitle_ID', 'Season', 'Episode_Num']
episode_df.fillna(random.randint(1, 100), inplace= True)
episode_df.replace('\\N', None, inplace=True)

episode_df.head()

,Title_ID,parentTitle_ID,Season,Episode_Num
0,tt0073481,tt0298685,1,4
1,tt0088283,tt0806901,20,8
2,tt0088656,tt0088655,1,2
3,tt0088657,tt0088655,1,3
4,tt0090780,tt0190210,1,3


Person table:

In [56]:
person_df = dataframes['name_basics'][['nconst', 'primaryName', 'birthYear', 'deathYear', 'img_url_asset']]
person_df.columns = ['Name_ID', 'Name', 'Birth_Year', 'Death_Year', 'Image']
person_df.fillna(random.randint(1, 100), inplace=True)
person_df.replace('\\N', None, inplace=True)

person_df.head()

/tmp/ipykernel_6222/1169295718.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person_df.fillna(random.randint(1, 100), inplace=True)
/tmp/ipykernel_6222/1169295718.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person_df.replace('\\N', None, inplace=True)


,Name_ID,Name,Birth_Year,Death_Year,Image
0,nm0000019,Federico Fellini,1920,1993,https://image.tmdb.org/t/p/{width_variable}/jH...
1,nm0000030,Audrey Hepburn,1929,1993,https://image.tmdb.org/t/p/{width_variable}/nb...
2,nm0000035,James Horner,1953,2015,https://image.tmdb.org/t/p/{width_variable}/g2...
3,nm0000047,Sophia Loren,1934,None,https://image.tmdb.org/t/p/{width_variable}/94...
4,nm0000053,Robert Mitchum,1917,1997,https://image.tmdb.org/t/p/{width_variable}/l0...


Participates_In Table:

In [57]:
dataframes['title_principals']['category'] = dataframes['title_principals']['category'].replace('\\N', '')
dataframes['title_principals']['job'] = dataframes['title_principals']['job'].replace('\\N', '')

dataframes['title_principals']['movie_job'] = dataframes['title_principals'].apply(lambda row: row['category'] + (',' + row['job'] if row['job'] != '' and row['job'] != row['category'] else ''), axis=1)

participates_in_df = dataframes['title_principals'][['tconst', 'nconst', 'ordering', 'movie_job', 'characters']]
participates_in_df.columns = ['Title_FK', 'Name_FK', 'Ordering', 'Job_Category', 'Character']
participates_in_df.fillna(random.randint(1, 100), inplace=True)
participates_in_df.replace('\\N', None, inplace=True)

participates_in_df.head()

/tmp/ipykernel_6222/3850331671.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participates_in_df.fillna(random.randint(1, 100), inplace=True)
/tmp/ipykernel_6222/3850331671.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participates_in_df.replace('\\N', None, inplace=True)


,Title_FK,Name_FK,Ordering,Job_Category,Character
0,tt0000929,nm0066941,1,actor,None
1,tt0000929,nm0170183,2,actor,None
2,tt0000929,nm0092290,3,actor,None
3,tt0000929,nm0093361,4,director,None
4,tt0000929,nm1902148,5,producer,None


Genre and Title_Genre:

In [58]:
unique_genres = set()
for genre_list in dataframes['title_basics']['genres'].dropna().str.split(','):
    unique_genres.update([genre.strip() for genre in genre_list])

genres_df = pd.DataFrame({'Genre': list(unique_genres)})
genres_df.replace('\\N', None, inplace=True)
genres_df.dropna(how='all', inplace=True)

print(genres_df)

title_genre_df = dataframes['title_basics'][['tconst', 'genres']]
title_genre_df.columns = ['Title_FK', 'Genre']
title_genre_df['Genre'] = title_genre_df['Genre'].str.split(',')
title_genre_df = title_genre_df.explode('Genre')
title_genre_df.fillna(random.randint(1, 100), inplace=True)
title_genre_df.replace('\\N', None, inplace=True)

title_genre_df.head()

          Genre
0     Animation
1          News
2           War
3     Biography
4         Crime
5         Drama
6        Action
7        Comedy
8         Sport
9      Thriller
10       Family
11      Romance
12        Music
13        Short
14        Adult
15      History
16      Musical
17  Documentary
18      Fantasy
19      Mystery
20      Western
22       Sci-Fi
23    Adventure
24       Horror


/tmp/ipykernel_6222/2329004846.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_genre_df['Genre'] = title_genre_df['Genre'].str.split(',')


,Title_FK,Genre
0,tt0000929,Comedy
0,tt0000929,Short
1,tt0000977,Short
2,tt0034841,Animation
2,tt0034841,Short


Now, we push all the processed data into our database:

In [59]:
import re

for var_name, value in globals().items():
    if isinstance(value, pd.DataFrame) and re.search('[a-zA-Z]', var_name):
        print(f"DataFrame '{var_name}' columns:")
        print(value.columns.tolist())
        print()

DataFrame 'title_df' columns:
['Title_ID', 'Original_Title', 'Type', 'Start_Year', 'End_Year', 'Runtime', 'Average_Rating', 'Votes', 'isAdult', 'IMAGE']

DataFrame 'alt_title_df' columns:
['Title_FK', 'Ordering', 'Title_AKA', 'Region']

DataFrame 'episode_df' columns:
['Title_ID', 'parentTitle_ID', 'Season', 'Episode_Num']

DataFrame 'person_df' columns:
['Name_ID', 'Name', 'Birth_Year', 'Death_Year', 'Image']

DataFrame 'participates_in_df' columns:
['Title_FK', 'Name_FK', 'Ordering', 'Job_Category', 'Character']

DataFrame 'genres_df' columns:
['Genre']

DataFrame 'title_genre_df' columns:
['Title_FK', 'Genre']



Insert into Title:

In [60]:
import pymysql
import os
from dotenv import load_dotenv

load_dotenv()

host = os.environ.get('DB_HOST')
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWD')
db = os.environ.get('DB_NAME')

print([host, user, password, db])

os.system(f"mysql -h {host} -u {user} -p{password} < {os.path.abspath('../ntuaflix_ddl.sql')}")

connection = pymysql.connect(host=host, user=user, password=password, db=db)

title_primary_keys = {}
with connection.cursor() as cursor:
    # Dealing with Titles first
    q = "INSERT INTO `Title` (Title_ID, Original_Title, Type, Start_Year, End_Year, Runtime, Average_Rating, Votes, isAdult, IMAGE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    for index, row in title_df.iterrows():
        tconst = row['Title_ID']
        original_title = row['Original_Title']
        type = row['Type']
        start_year = row['Start_Year']
        end_year = row['End_Year']
        runtime = row['Runtime']
        average_rating = row['Average_Rating']
        votes = row['Votes']
        is_adult = row['isAdult']
        image = row['IMAGE']

        cursor.execute(q, (tconst, original_title, type, start_year, end_year, runtime, average_rating, votes, is_adult, image))
        connection.commit()
        print("cursor commited")
    
        # Saving the primary key
        title_primary_keys[tconst] = cursor.lastrowid

['localhost', 'root', 'password', 'tl']
cursor commited
cursor commited
cursor commited


ERROR 1075 (42000) at line 100: Incorrect table definition; there can be only one auto column and it must be defined as a key


cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor c

cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor c

Insert into Person:

In [61]:
person_primary_keys = {}

with connection.cursor() as cursor:
    query = "INSERT INTO `Person` (Name_ID, Name, Image, Birth_Year, Death_Year) VALUES (%s, %s, %s, %s, %s)"
    
    for index, row in person_df.iterrows():
        name_id = row['Name_ID']
        name = row['Name']
        image = row['Image']
        birth_year = row['Birth_Year']
        death_year = row['Death_Year']

        try:
            cursor.execute(query, (name_id, name, image, birth_year, death_year))
            connection.commit()
        
            # Saving the auto-generated primary key
            person_primary_keys[name_id] = cursor.lastrowid
            print(f"Inserted {name_id} with PK: {cursor.lastrowid}")
        except Exception as e:
            print(f"Error inserting {name_id}: {e}")
            connection.rollback()  # Rollback in case of error


Inserted nm0000019 with PK: 1
Inserted nm0000030 with PK: 2
Inserted nm0000035 with PK: 3
Inserted nm0000047 with PK: 4
Inserted nm0000053 with PK: 5
Inserted nm0000085 with PK: 6
Inserted nm0000095 with PK: 7
Inserted nm0000099 with PK: 8
Inserted nm0000123 with PK: 9
Inserted nm0000147 with PK: 10
Inserted nm0000149 with PK: 11
Inserted nm0000154 with PK: 12
Inserted nm0000167 with PK: 13
Inserted nm0000176 with PK: 14
Inserted nm0000177 with PK: 15
Inserted nm0000198 with PK: 16
Inserted nm0000199 with PK: 17
Inserted nm0000214 with PK: 18
Inserted nm0000221 with PK: 19
Inserted nm0000248 with PK: 20
Inserted nm0000256 with PK: 21
Inserted nm0000259 with PK: 22
Inserted nm0000275 with PK: 23
Inserted nm0000293 with PK: 24
Inserted nm0000296 with PK: 25
Inserted nm0000334 with PK: 26
Inserted nm0000347 with PK: 27
Inserted nm0000355 with PK: 28
Inserted nm0000372 with PK: 29
Inserted nm0000374 with PK: 30
Inserted nm0000375 with PK: 31
Inserted nm0000382 with PK: 32
Inserted nm000038

Insert into Participates_In:

In [62]:
with connection.cursor() as cursor:
    query = "INSERT INTO `Participates_In` (Title_FK, Name_FK, Ordering, Job_Category, `Character`) VALUES (%s, %s, %s, %s, %s)"
    
    for index, row in participates_in_df.iterrows():
        title_fk = title_primary_keys.get(row['Title_FK'])
        print(title_fk)
        name_fk = person_primary_keys.get(row['Name_FK'])
        print(name_fk)
        ordering = row['Ordering']
        job_category = row['Job_Category']
        character = row['Character']

        if title_fk is not None and name_fk is not None:
            cursor.execute(query, (title_fk, name_fk, ordering, job_category, character))
            connection.commit()
        else:
            print("an error occured")


1
None
an error occured
1
None
an error occured
1
595
1
None
an error occured
1
3789
1
2670
2
595
2
3789
2
2670
51
810
51
None
an error occured
52
3747
52
3260
52
3327
52
1001
52
2609
52
2232
52
3758
52
2446
52
1200
52
285
3
2361
53
1287
53
None
an error occured
53
None
an error occured
53
None
an error occured
53
None
an error occured
53
None
an error occured
53
532
53
261
53
None
an error occured
53
1210
4
20
4
2448
4
2510
4
2343
4
1360
4
3294
4
3493
4
1297
54
2146
54
1678
54
2945
54
3830
54
2261
54
737
54
2736
54
557
54
1530
54
1411
55
1689
55
3658
55
3538
55
1997
55
1488
55
1560
55
3042
55
2528
55
1513
55
3015
56
627
56
1307
56
2215


56
1117
56
3609
56
3359
56
1856
56
3169
56
3744
56
2244
57
589
57
853
57
3190
57
3191
57
1199
57
2013
57
265
57
232
57
3592
58
2117
58
3082
58
3852
58
3850
58
2587
58
3844
58
2697
58
3853
59
2074
59
3867
59
3932
59
377
59
2926
59
2037
59
3665
59
743
59
2711
351
50
351
2532
351
145
351
996
351
462
351
2970
351
2759
351
1477
60
3682
60
2145
60
895
60
2426
60
3909
60
3243
60
3679
60
3907
60
2235
60
1967
5
3
5
1624
5
1159
5
2868
5
2533
5
1486
5
2583
5
2715
5
1931
5
2788
61
144
61
844
61
106
61
3569
61
990
61
1316
61
989
61
2323
62
1707
63
242
63
70
63
3630
63
138
63
137
63
994
63
924
63
3271
63
2200
63
257
64
2217
65
1384
65
128
65
2701
65
1309
65
1941
65
2915
65
1074
65
1733
65
2758
65
957
6
915
6
416
6
1541
6
795
6
2847
6
2571
6
3540
6
1756
7
3733
7
3707
7
3813
7
3812
7
32
66
2206
67
1145
67
2449
67
1723
67
1721
67
3311
67
891
67
2998
67
3820
67
3821
67
3809
8
1330
8
3959
8
805
8
1026
8
32
8
2886
8
2392
8
3811
8
3756
68
3835
68
871
68
3565
68
862
68
2089
68
842
68
3912
68
3922
68
3787
68

Genre and Title_Genre:

In [63]:
with connection.cursor() as cursor:
    genre_query = "INSERT INTO `Genre` (Genre) VALUES (%s)"
    genre_primary_keys = {}

    print(genres_df)
    for index, row in genres_df.iterrows():
        genre = row['Genre']
        cursor.execute(genre_query, (genre,))
        connection.commit()

        # Saving the primary key
        genre_primary_keys[genre] = cursor.lastrowid

    title_genre_query = "INSERT INTO `Title_Genre` (Title_FK, Genre_FK) VALUES (%s, %s)"

    for index, row in title_genre_df.iterrows():
        title_fk = title_primary_keys.get(row['Title_FK'])
        if title_fk == None:
            continue
        genre_fk = genre_primary_keys.get(row['Genre'])


        try:
            if title_fk is not None and genre_fk is not None:
                cursor.execute(title_genre_query, (title_fk, genre_fk))
                connection.commit()
                print("sth")
            else:
                connection.rollback()
        except Exception as e:
            print(str(e))

          Genre
0     Animation
1          News
2           War
3     Biography
4         Crime
5         Drama
6        Action
7        Comedy
8         Sport
9      Thriller
10       Family
11      Romance
12        Music
13        Short
14        Adult
15      History
16      Musical
17  Documentary
18      Fantasy
19      Mystery
20      Western
22       Sci-Fi
23    Adventure
24       Horror


sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth
sth


Episodes

In [64]:
episode_query = "INSERT INTO `Episodes` (Title_FK, Parent_Title_FK, Season, Episode_Num) VALUES (%s, %s, %s, %s)"

with connection.cursor() as cursor:
    for index, row in episode_df.iterrows():
        title_fk = title_primary_keys.get(row['Title_ID'])
        parent_title_fk = title_primary_keys.get(row['parentTitle_ID'])
        print(parent_title_fk)
        season = row['Season']
        episode_num = row['Episode_Num']

        # Ensure both foreign keys are found
        if title_fk is not None and parent_title_fk is not None:
            try:
                cursor.execute(episode_query, (title_fk, parent_title_fk, season, episode_num))
                connection.commit()
            except Exception as e:
                print(f"Error inserting episode: {e}")
                connection.rollback()


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Alternative Title

In [65]:
alt_title_primary_keys = {}
count = 0

with connection.cursor() as cursor:
    query = "INSERT INTO `Alt_Title` (Title_FK, Ordering, Title_AKA, Region) VALUES (%s, %s, %s, %s)"
    
    for index, row in alt_title_df.iterrows():
        title_fk = title_primary_keys.get(row['Title_FK'])
        ordering = row['Ordering']
        title_aka = row['Title_AKA']
        region = row['Region']
        count += 1

        try:
            cursor.execute(query, (title_fk, ordering, title_aka, region))
            connection.commit()
        
            # Saving the auto-generated primary key
            alt_title_primary_keys[(title_fk, title_aka)] = cursor.lastrowid
            print(f"Inserted Title_FK: {title_fk}, Title_AKA: {title_aka} with PK: {cursor.lastrowid}")
        except Exception as e:
            print(f"Error inserting Title_FK: {title_fk}, Title_AKA: {title_aka}: {e}")
            connection.rollback()  # Rollback in case of error


Inserted Title_FK: 1, Title_AKA: Willys Streiche: Klebolin klebt alles with PK: 1
Inserted Title_FK: 1, Title_AKA: Klebolin klebt alles with PK: 2
Inserted Title_FK: 1, Title_AKA: Klebolin klebt alles with PK: 3
Inserted Title_FK: 2, Title_AKA: Mutterliebe with PK: 4
Inserted Title_FK: 2, Title_AKA: Mutterliebe with PK: 5
Inserted Title_FK: 51, Title_AKA: La tierra de los toros with PK: 6
Inserted Title_FK: 51, Title_AKA: La tierra de los toros with PK: 7
Inserted Title_FK: 51, Title_AKA: La terre des taureaux with PK: 8
Inserted Title_FK: 51, Title_AKA: The Land of the Bulls with PK: 9
Inserted Title_FK: 51, Title_AKA: La Terre des taureaux with PK: 10
Inserted Title_FK: 52, Title_AKA: Dama de noche with PK: 11
Inserted Title_FK: 52, Title_AKA: Dama de noche with PK: 12
Inserted Title_FK: 52, Title_AKA: Éjszakai gyilkosság with PK: 13
Inserted Title_FK: 52, Title_AKA: Lady of the Night with PK: 14
Inserted Title_FK: 3, Title_AKA: Hen Hop with PK: 15
Inserted Title_FK: 3, Title_AKA: He

Inserted Title_FK: 61, Title_AKA: A Touch of the Sun with PK: 65
Inserted Title_FK: 61, Title_AKA: No Secrets! with PK: 66
Inserted Title_FK: 62, Title_AKA: Kommen und gehen with PK: 67
Inserted Title_FK: 62, Title_AKA: Kommen und gehen with PK: 68
Inserted Title_FK: 63, Title_AKA: Me and the Kid with PK: 69
Inserted Title_FK: 63, Title_AKA: Kid et le Truand with PK: 70
Inserted Title_FK: 63, Title_AKA: Taking Gary Feldman with PK: 71
Inserted Title_FK: 63, Title_AKA: Me and the Kid with PK: 72
Inserted Title_FK: 63, Title_AKA: Harry e Carota with PK: 73
Inserted Title_FK: 63, Title_AKA: Eu E Meu Amigo with PK: 74
Inserted Title_FK: 63, Title_AKA: Я и ребёнок with PK: 75
Inserted Title_FK: 63, Title_AKA: Atrévete a secuestrar un niño y verás with PK: 76
Inserted Title_FK: 63, Title_AKA: Me and the Kid with PK: 77
Inserted Title_FK: 63, Title_AKA: Hurra! Jag är kidnappad with PK: 78
Inserted Title_FK: 63, Title_AKA: Me and the Kid with PK: 79
Inserted Title_FK: 63, Title_AKA: Mein Freun